<a href="https://colab.research.google.com/github/zhangxs131/Text_Classification/blob/main/run_glue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets

     |████████████████████████████████| 4.0 MB 5.2 MB/s 
     |████████████████████████████████| 325 kB 45.1 MB/s 
     |████████████████████████████████| 880 kB 43.4 MB/s 
     |████████████████████████████████| 77 kB 6.7 MB/s 
     |████████████████████████████████| 6.6 MB 40.0 MB/s 
     |████████████████████████████████| 596 kB 27.6 MB/s 
     |████████████████████████████████| 1.1 MB 38.1 MB/s 
     |████████████████████████████████| 212 kB 54.3 MB/s 
     |████████████████████████████████| 136 kB 51.9 MB/s 
     |████████████████████████████████| 127 kB 49.2 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 
     |████████████████████████████████| 271 kB 51.4 MB/s 
     |████████████████████████████████| 144 kB 49.7 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=3c9fb18a75bb37553be91a38328c7886a87907b8933e4d847d860f1e5288a53c
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d66029

In [4]:
from datasets import load_dataset
import pandas as pd


mrpc=load_dataset('SetFit/mrpc')
train_df=pd.DataFrame(mrpc['train'][:2000])
valid_df=pd.DataFrame(mrpc['validation'][:200])

train_df=train_df.drop(columns=['idx','label_text'])
valid_df=valid_df.drop(columns=['idx','label_text'])

train_df.rename(columns={'text1':'sentence1','text2':'sentence2'})
valid_df.rename(columns={'text1':'sentence1','text2':'sentence2'})

train_df.to_csv('train.csv',index=False)
valid_df.to_csv('valid.csv',index=False)

Using custom data configuration SetFit--mrpc-2a21aa905b91701f
Reusing dataset json (/root/.cache/huggingface/datasets/json/SetFit--mrpc-2a21aa905b91701f/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
!git clone https://github.com/huggingface/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 93339, done.
remote: Counting objects: 100% (224/224), done.
remote: Compressing objects: 100% (148/148), done.
remote: Total 93339 (delta 100), reused 157 (delta 58), pack-reused 93115
Receiving objects: 100% (93339/93339), 85.89 MiB | 25.08 MiB/s, done.
Resolving deltas: 100% (68385/68385), done.


In [5]:
!python transformers/examples/pytorch/text-classification/run_glue.py \
  --model_name_or_path bert-base-cased \
  --train_file train.csv \
  --validation_file valid.csv\
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 16 \
  --fp16 \
  --learning_rate 2e-5 \
  --num_train_epochs 2 \
  --output_dir /tmp/mrpc/

05/05/2022 11:26:43 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
05/05/2022 11:26:43 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=Fals

In [16]:
from transformers import pipeline
from transformers import AutoTokenizer,AutoModelForSequenceClassification
import torch

tokenizer=AutoTokenizer.from_pretrained('mrpc')
model=AutoModelForSequenceClassification.from_pretrained('mrpc')


s1='I love you.'
s2="I love you so much."

input_id=tokenizer(s1,s2,return_tensors='pt')
output=model(**input_id).logits

result=torch.softmax(output,dim=1).tolist()[0]

# generate=pipeline(task='sentiment-analysis',model=model,tokenizer=tokenizer)
# result=generate([s1,s2])
result

[0.6966950297355652, 0.3033050298690796]

##感觉这个就有点不对，我fine-tune的数据集就是mrpc两者是否可以解释，但结果却都是0大，1小。